In [1]:
import torch

from torch.utils.data import DataLoader

from utils.load import MetaLoader, DialogueTrainLoader
from utils.preprocess import Preprocessor, Augmentation
from utils.encoder import Encoder

from data import FH2024Dataset, collate_fn
from net.tokenizer import SubWordEmbReaderUtil
from net.model import Model

In [2]:
swer_path = './sstm_v0p5_deploy/sstm_v4p49_np_n36134_d128.dat'
swer = SubWordEmbReaderUtil(swer_path)

meta_path = '../../datasets/FH_2024/subtask3/mdata.wst.txt.2023.08.23'
meta_loader = MetaLoader(path=meta_path, swer=swer)
img2id, id2img, img_similarity = meta_loader.get_dataset()

In [3]:
train_path = "../../datasets/FH_2024/subtask3/task1.ddata.wst.txt"
train_diag_loader = DialogueTrainLoader(path=train_path)
train_raw_dataset = train_diag_loader.get_dataset()

In [4]:
train_raw_dataset[0]

{'description': ['안녕_하 세 요 코디 봇 입 니다 무엇 을 도와 드릴_까 요',
  '최근 에 열린 꽃 축제 에 가_려고 하_는데 그때 입 을 스커트 를 포함_한 의상 추천_해 주 세 요',
  '원하 시 는 스커트 기장 이 있 으신가 요',
  '중간 기장 으로 보여_주 세 요',
  '겉옷 이 포함_된 코디로 추천_해 드릴_까 요',
  '얇 은 가디건 으로 추천 부탁 드려_요',
  '네 반영 하_여 추천 드리 겠 습니다 잠시 만 기다려_주 세 요',
  '아이보리 색상 의 머메이드 형 스커트 와 부드러운 소재 의 베이지 색상 가디건 을 포함_한 코디 를 추천_해 드립 니다 마음 에 드 시_나 요',
  '상의 와 신발 은 캐쥬얼 한 디자인 이 마음 에 들_어 요 그런데 가디건 은 길이 가 조금 긴 것 같_아 짧 은 의상 으로 치마 는 활동_하 기 편한 스커트 로 부탁 드려_요',
  '네 가디건 은 원하 시 는 색상 이 있_나 요',
  '베이지색 계열 로 보여_주 세 요',
  '네 반영 하_여 다시 추천 드리 겠 습니다 잠시 만 기다려_주 세 요',
  '퍼프 소매 디자인 이 가미 된 베이지 색상 의 가디건 과 넛 넛 한 핏 으로 활동_하 기 편한 플레어 형 스커트 를 추천_해 드립 니다 마음 에 드 시_나 요',
  '가디건 은 핏 이 마음 에 드_는데 스커트 는 때 가 탈 것 같_아 다른 색상 으로 보여_주 세 요',
  '네 어두운 색상 으로 다시 추천_해 드리 겠 습니다 잠시 만 기다려_주 세 요',
  '종아리 까지 오 는 블랙 색상 의 스커트 입 니다 마음 에 드 시_나 요',
  '색상 이랑 디자인 이 튀 지_않 고 움직이 기 편해 보여 마음 에 들_어 요',
  '마음 에 드 셨_다 니 다행 입 니다',
  '선택_하 신 아이템 으로 구성_된 최종 코디 입니다 마음 에 드 시_나 요',
  '네 스타일 에 신경_쓰 면서_도 간단_하 게 입_고 갈_수 있 을 것 같_아 마음 에 들_어 요',
  '마음 에 드 셨_다 니 

In [5]:
preprocessor = Preprocessor(num_rank=3, num_coordi=4, top_k=50)
train_dataset = preprocessor(train_raw_dataset, img2id, id2img, img_similarity)

In [6]:
len(train_dataset)

995

In [7]:
augmentation = Augmentation(num_aug=5, num_rank=3, num_coordi=4, top_k=50)
train_dataset = augmentation(train_dataset, img2id, id2img, img_similarity)

In [8]:
len(train_dataset)

5970

In [9]:
encoder = Encoder(swer=swer, img2id=img2id, num_coordi=4, mem_size=16, meta_size=4)
encoded_train_dataset = encoder(train_dataset)

In [10]:
# -- Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -- Training
BATCH_SIZE = 16
EPOCH = 10

# -- Model
KEY_SIZE = 300
MEM_SIZE = 16
HOPS = 3
EVAL_NODE = '[6000,6000,6000,200][2000,2000]'
DROP_PROB = 0.1

In [11]:
train_dataset = FH2024Dataset(dataset=encoded_train_dataset)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, collate_fn=collate_fn)

In [12]:
for batch in train_loader:
    desc = batch['description'].to(device)
    coordi = batch['coordi'].to(device)
    rank = batch['rank'].to(device)
    break

print(f"desc: {desc.shape}, coordi: {coordi.shape}, rank: {rank.shape}")

desc: torch.Size([16, 16, 128]), coordi: torch.Size([16, 3, 2048]), rank: torch.Size([16])


In [13]:
item_size = [len(img2id[i]) for i in range(4)] # 각 카테고리별 개수
net = Model(emb_size=swer.get_emb_size(), 
            key_size=KEY_SIZE, 
            mem_size=MEM_SIZE,
            meta_size=4, 
            hops=HOPS, 
            item_size=item_size, 
            coordi_size=4,
            eval_node=EVAL_NODE, 
            num_rnk=3, 
            use_batch_norm=False, 
            use_dropout=True,
            zero_prob=DROP_PROB,
            use_multimodal=False,
            img_feat_size=4096).to(device)

In [14]:
logits = net(desc, coordi)
logits.shape

torch.Size([16, 6])

In [15]:
import torch.nn as nn
criterion = nn.CrossEntropyLoss()

loss = criterion(logits, rank)
print(loss)

tensor(1.7947, device='cuda:0', grad_fn=<NllLossBackward0>)
